In [10]:
import os
from openai import OpenAI, AsyncClient
import asyncio
key = ''
url = "https://deepseek.perfxlab.cn/v1" 

In [11]:
with open('../prompt/role_play_kimi.txt','r') as f:
    prompt = f.read()

with open('../Output/人设v1.0/黍.md','r') as f:
    rolecard = f.read()

message = [  
        {"role":"system","content":prompt.format(rolecard)},
        ]

In [13]:
import datetime
datetime.datetime.now().strftime('%c')

In [14]:
##  test
client = OpenAI(
    base_url = url,
    api_key = key
)

def test_chat(msg, history):

    history.append({"role": "user", "content": msg})
    res = client.chat.completions.create(
    model="DeepSeek-V3", # 指定要使用的模型
    messages = history, # 用户消息，包含用户的输入
    temperature=0.7, 
    max_tokens=1024, # 限制生成的最大令牌数（单词数量）
    n=1, # 生成的回复数量
    presence_penalty=0.3, # 控制新话题出现的惩罚，值越高，新话题出现越少
    frequency_penalty=0.3, # 控制重复话语的惩罚，值越高，重复率越低
    stream=False, # 启用流式响应，允许逐块接收生成内容
)
    history.append({"role":"assistant","content":res.choices[0].message.content})
    print(history[-1]['content'])
    return res, history



history = message.copy()

In [ ]:
res, history = test_chat('你好', history)

In [ ]:
## 一轮  batchtest

aclient = AsyncClient(
    base_url = url,
    api_key = key
)

async def acreate(massgae):
    res = await aclient.chat.completions.create(
    model="DeepSeek-V3",
    messages = massgae,
    temperature=0.9, 
    max_tokens=1024, 
    n=1, 
    presence_penalty=0.3, 
    frequency_penalty=0.5, 
    stream=False,
    )
    return res

async def test_chat_batch(msg, history):
    tasks = [acreate(history + [{"role": "system", "content": f"## 当前时间: \n{t}"},{"role": "user", "content": m}]) for m,t in msg]
    results = await asyncio.gather(*tasks)
    return results

test_call = [
    ("在吗？", "星期日 2025-3-9 12:30"),
    ("你今晚有空吗？", "星期六 2025-3-8 14:00"),
    ("什么时候才能放假啊","星期四 2025-3-20 15:30"),
    ("", "星期一 2025-3-1 1:20")
]


In [18]:
res = await test_chat_batch(test_call, message)
for i in res:
    print(i.choices[0].message.content)

博士，我在这里。今日阳光和煦，正适合在田间劳作。您找我可是有事相商？
博士，今夜月色正好，我本想在田埂上散步，看那稻穗在月光下摇曳。不过既然您找我，想必是有要事相商吧？只要不是与弟弟那孩子气的新游戏有关就好。
博士，今日正值春分时节，万物复苏之际。劳逸结合固然重要，但此刻的每一份努力，都是为将来的丰收播下希望的种子。正如农人耕耘不辍，方能期待金秋的硕果累累。让我们携手前行，待到风调雨顺、五谷丰登之日，便是我们安享闲暇之时。相信在这段时光里所付出的辛劳，终将化作美好的回忆与收获的喜悦。
博士，夜深了，您还未歇息吗？（轻声细语地）窗外的月光洒在麦田上，如同银色的波浪，柔美而宁静。在这样的夜晚，最适合放下心中的重担，与自然对话。或许您有些心事？不妨与我分享，让我为您分担一二。
